# Parameter Efficient Fine Tuning: Adapter Layers on Financial Sentiment Analysis


**Adapter fine-tuning** is a technique used to modify and specialize pre-trained models like large language models (LLMs) without retraining the entire model. Instead of updating all the parameters, small, additional modules called **adapters** are inserted into the model. During training, only the adapters are updated, leaving the rest of the model's parameters frozen. This approach is computationally efficient and reduces the risk of overfitting.

To reduce bias, we can fine-tune adapters with bias-mitigated datasets or targeted interventions. For instance:

1. **Bias-Corrected Data**: Train the adapters using datasets curated to reduce stereotypical or biased patterns.
2. **Counterfactual Data Augmentation**: Use data with rewritten examples to address specific biases (e.g., gender, race).
3. **Regularization Techniques**: Apply constraints or loss functions to penalize biased outputs during fine-tuning.

When fine tune an AI model, the output model might produce less-biased responses. However, it is basically masking and safeguarding a model, rather than resolving the root cause. So, introducing the details of fine tuning mechanism is essential for a fairness-oriented record keeping. 

- Read a more detailed overview on fine-tuning using adapter layers: <https://arxiv.org/abs/2303.15647>
- Also, you can read PEFT on Huggingface: <https://huggingface.co/blog/peft> 

Considering these details, a fairness report should include:

- [ ] Which design decisions/parameters are important for other stakeholders?
- [ ] Which details should be added to the evaluation results?

In [ ]:
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_model,
    PeftType,
    PromptTuningConfig,
)

import evaluate
from transformers import BertTokenizer,BertForSequenceClassification, BertConfig, get_linear_schedule_with_warmup
from tqdm import tqdm

In this notebook, we will use Indian Financial News dataset (<https://huggingface.co/datasets/kdave/Indian_Financial_News>) to improve the model's performance of analysing news sources from different geographies. In our previous experiments, we observed that 0.88 accuracy of FinBERT model performance reduced to 0.57 in three category classification task. The dataset includes equal number of samples (n=8987) from each tone category "Positive", "Negative" and "Neutral". Fine-tuning using a real-word, different country-sourced dataset is important for fairness to improve model's performance on different technical jargon, currency, and other implicit bias sources. 

In [ ]:
from datasets import load_dataset

ds = load_dataset("kdave/Indian_Financial_News", split="train")

In [ ]:
ds = ds.train_test_split(test_size=0.2)

In [ ]:
# Remove URL and Content columns
ds = ds.remove_columns(["URL", "Content"])

# Sentiment column encode labels "Positive" as 2, "Neutral" as 1, and "Negative" as 0
ds = ds.map(
    lambda x: {"Sentiment": 2 if x["Sentiment"] == "Positive" else 1 if x["Sentiment"] == "Neutral" else 0}
)

In [ ]:
ds["train"]

In [ ]:
ds["test"]

### Model and Data Tokenization

In [ ]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
config = BertConfig.from_pretrained('yiyanghkust/finbert-tone')

In [ ]:
batch_size = 32
task = "mrpc"
peft_type = PeftType.PROMPT_TUNING
num_epochs = 20

In [ ]:
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples["Summary"], max_length=None, truncation=True, padding="max_length")
    return outputs

tokenized_datasets = ds.map(
    tokenize_function,
    batched=True,
    remove_columns=["Summary"],
)

tokenized_datasets = tokenized_datasets.rename_column("Sentiment", "labels")

def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)

### Add adapter layers

In [ ]:
peft_config = PromptTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=10)
lr = 1e-3

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
len(train_dataloader)

In [ ]:
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
metric = evaluate.load("accuracy")

for epoch in range(1):
#for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

In [ ]:
model.save_pretrained("./model/finbert-peft-prompt-tuning", from_pt=True)

## Use the Model

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer

peft_model_id = "./model/finbert-peft-prompt-tuning"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
inference_model = PeftModel.from_pretrained(inference_model, peft_model_id)

inference_model.to(device)
inference_model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = inference_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )

eval_metric = metric.compute()
print(eval_metric)